# ELephantSQL Database Builder:

A script for importing local data into a database so that my web app can access it

In [ ]:
import json
import sqlalchemy
import pandas as pd



Connecting to ElephantSQL database:

In [ ]:
with open ("..\key_paths\path_to_keys_folder.txt") as file:
    key_path = file.read()

In [ ]:
with open(key_path+"\\elephantsql_kjb3webchartapp_db_url.txt") as file:
    db_url = file.read()
    # This code reads in my database's URL. This URL is listed on the home page for my database within elephantsql.com. As shown below, SQLAlchemy can use this URL to connect to the database. 

elephantsql_db_url_for_sqlalchemy = db_url.replace('postgres://', 'postgresql://')
# This change, which is required for SQLAlchemy to work correctly, is based on the code suggested at:
# # https://help.heroku.com/ZKNTJQSK/why-is-sqlalchemy-1-4-x-not-connecting-to-heroku-postgres

In [ ]:
elephantsql_engine = sqlalchemy.create_engine(elephantsql_db_url_for_sqlalchemy)

## Loading in data

Next, I'll load in a .csv file that was created using the [flights_table_builder](https://github.com/kburchfiel/route_maps_builder/blob/master/flights_table_builder.ipynb) script within my [Route Maps Builder](https://github.com/kburchfiel/route_maps_builder) program. The original file is too large to host on GitHub, but it can be found within [this Google Drive page](https://drive.google.com/file/d/1gUozlxpVIO-gQ6d-AQ4Qgpg91Ut3kqUD/view?usp=sharing).



In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
df_flights = pd.read_csv(r'C:\Users\kburc\D1V1\Documents\!Dell64docs\Programming\py\kjb3_programs\route_maps_builder\routes_planes_coordinates_for_mapping_2018.csv')
df_flights.drop('Unnamed: 0', axis = 1,inplace=True)
df_flights['UNIQUE_CARRIER_NAME'].fillna("Unknown", inplace = True) # This line will allow rows without an airline name to still be processed by pivot table operations that reference those names.
df_flights

## Pivoting and filtering data

This table currently contains around 350,000 rows and 65 columns. However, my web app doesn't need all of those rows present in its own PostgreSQL database; instead, it just needs enough data to generate various charts. Therefore, I'll now create some modified versions of this data and then upload it to the PostgreSQL database.

In [ ]:
df_airline_origin_dest_plane_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER_NAME', 'ORIGIN_DEST', 'Plane_Type_Text'], values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False)
df_airline_origin_dest_plane_pivot = df_airline_origin_dest_plane_pivot.query("PASSENGERS > 100000").copy()
df_airline_origin_dest_plane_pivot['AIRPORT 1'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[0]
df_airline_origin_dest_plane_pivot['AIRPORT 2'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[1]
df_airline_origin_dest_plane_pivot.reset_index(drop=True,inplace=True)
df_airline_origin_dest_plane_pivot

The following cell exports this pivot table to .csv format locally (so that it can be accessed without a database connection) and to .sql format online.

In [ ]:
df_airline_origin_dest_plane_pivot.to_csv('local_copy_of_airports_airlines_aircraft_2018.csv', index = False)

df_airline_origin_dest_plane_pivot.to_sql("airports_airlines_aircraft_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

I believe the output of the above cell shows the smallest 3 digits of the number of rows updated within the table. Thus, if 2,804 rows were created within the table, the output will be 804.

The following cell reads the new database table back into Pandas in order to verify that it was imported successfully:

In [ ]:
pd.read_sql("airports_airlines_aircraft_2018", con = elephantsql_engine)

## Creating additional tables for the database:

In [ ]:
df_top_20_airports = df_flights.pivot_table(index = 'destination_iata_code', values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).head(20) # Calculates the top 20 airports by arriving passenger count.
df_top_20_airports.reset_index(inplace=True)
df_top_20_airports['Rank'] = df_top_20_airports['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airports.rename(columns={"destination_iata_code":"Airport","PASSENGERS":"2018_Passenger_Arrivals"},inplace=True)
df_top_20_airports

In [ ]:
df_top_20_airports.to_csv('top_20_airports_by_pax_arrivals_2018.csv', index = False)

df_top_20_airports.to_sql("top_20_airports_by_pax_arrivals_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

In [ ]:
df_top_20_airlines = df_flights.pivot_table(index = 'UNIQUE_CARRIER_NAME', values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False).head(20).reset_index(drop=True)
df_top_20_airlines['Rank'] = df_top_20_airlines['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airlines.rename(columns={"UNIQUE_CARRIER_NAME":"Airline","PASSENGERS":"Passengers"},inplace=True)
df_top_20_airlines

In [ ]:
df_top_20_airlines.to_csv('top_20_airlines_by_passengers_2018.csv', index = False)
df_top_20_airlines.to_sql("top_20_airlines_by_passengers_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

In [ ]:
df_airport_airline_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER_NAME', 'destination_iata_code'], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).query("PASSENGERS > 0").reset_index()
df_airport_airline_pivot.rename(columns={"UNIQUE_CARRIER_NAME":"Airline","destination_iata_code":"Dest_Airport","PASSENGERS":"Passengers"},inplace=True)
df_airport_airline_pivot

In [ ]:
df_airport_airline_pivot['Passengers'].sum()

In [ ]:
df_flights['PASSENGERS'].sum()

In [ ]:
df_airport_airline_pivot.to_csv('airport_airline_pairs_2018.csv', index = False)
df_airport_airline_pivot.to_sql('airline_airport_pairs_2018', if_exists = 'replace', con = elephantsql_engine, index = False)